# Download Dataset

In [ ]:
import pandas as pd
import requests
import os

# Hàm helper để download từ Google Drive
def download_from_drive(drive_url, local_path):
    response = requests.get(drive_url)
    response.raise_for_status()
    with open(local_path, 'wb') as f:
        f.write(response.content)

urls = {
    "train": {
        "sentences": "https://drive.google.com/uc?id=1nzak5OkrheRV1ltOGCXkT671bmjODLhP&export=download",
        "sentiments": "https://drive.google.com/uc?id=1ye-gOZIBqXdKOoi_YxvpT6FeRNmViPPv&export=download",
        "topics":     "https://drive.google.com/uc?id=14MuDtwMnNOcr4z_8KdpxprjbwaQ7lJ_C&export=download",
    },
    "validation": {
        "sentences": "https://drive.google.com/uc?id=1sMJSR3oRfPc3fe1gK-V3W5F24tov_517&export=download",
        "sentiments": "https://drive.google.com/uc?id=1GiY1AOp41dLXIIkgES4422AuDwmbUseL&export=download",
        "topics":     "https://drive.google.com/uc?id=1DwLgDEaFWQe8mOd7EpF-xqMEbDLfdT-W&export=download",
    },
    "test": {
        "sentences": "https://drive.google.com/uc?id=1aNMOeZZbNwSRkjyCWAGtNCMa3YrshR-n&export=download",
        "sentiments": "https://drive.google.com/uc?id=1vkQS5gI0is4ACU58-AbWusnemw7KZNfO&export=download",
        "topics":     "https://drive.google.com/uc?id=1_ArMpDguVsbUGl-xSMkTF_p5KpZrmpSB&export=download",
    }
}

def prepare_split(split_name, urls_for_split, output_dir="data"):
    os.makedirs(output_dir, exist_ok=True)
    paths = {}
    for kind, url in urls_for_split.items():
        local_path = os.path.join(output_dir, f"{split_name}_{kind}.txt")
        if not os.path.exists(local_path):
            print(f"Downloading {split_name} {kind} …")
            download_from_drive(url, local_path)
        else:
            print(f"File {local_path} đã tồn tại, bỏ qua download.")
        paths[kind] = local_path

    # Đọc file mỗi dòng là một bản ghi
    # Dùng read_csv với delimiter mặc định (phân cách theo dấu phẩy nếu có),
    # ở đây file chỉ có một cột, nên ta chỉ cần đọc cả dòng là string
    def read_single_column_txt(path, column_name):
        # Dùng read_csv, mỗi dòng là một record
        return pd.read_csv(path, header=None, names=[column_name], dtype=str, encoding="utf-8", sep="\r\n", engine="python")

    # Nếu cách trên vẫn lỗi, dùng cách fallback: đọc thủ công với Python
    def read_single_column_manual(path, column_name):
        with open(path, 'r', encoding='utf-8') as f:
            lines = [line.strip("\n") for line in f]
        return pd.DataFrame({column_name: lines})

    # Thử đọc
    try:
        df_sent = read_single_column_txt(paths["sentences"], "sentence")
    except Exception as e:
        print("Đọc sentences bằng read_csv với sep=\"\\r\\n\" bị lỗi, dùng manual:", e)
        df_sent = read_single_column_manual(paths["sentences"], "sentence")

    try:
        df_senti = read_single_column_txt(paths["sentiments"], "sentiment")
    except Exception as e:
        print("Đọc sentiments bị lỗi, dùng manual:", e)
        df_senti = read_single_column_manual(paths["sentiments"], "sentiment")

    try:
        df_topic = read_single_column_txt(paths["topics"], "topic")
    except Exception as e:
        print("Đọc topics bị lỗi, dùng manual:", e)
        df_topic = read_single_column_manual(paths["topics"], "topic")

    # Kiểm tra số dòng
    assert len(df_sent) == len(df_senti) == len(df_topic), \
        f"Số dòng không khớp ở split {split_name}: sentences {len(df_sent)}, sentiments {len(df_senti)}, topics {len(df_topic)}"

    df = pd.concat([df_sent, df_senti, df_topic], axis=1)

    output_csv = os.path.join(output_dir, f"{split_name}.csv")
    print(f"Lưu file CSV: {output_csv}")
    df.to_csv(output_csv, index=False, encoding="utf-8")

    return df

def download_and_prepare_all(output_dir="data"):
    datasets = {}
    for split, split_urls in urls.items():
        df = prepare_split(split, split_urls, output_dir=output_dir)
        datasets[split] = df
    return datasets

if __name__ == "__main__":
    datasets = download_and_prepare_all(output_dir="uit_vsf_feedback_data")
    # Kết quả: có các file train.csv, validation.csv, test.csv trong thư mục


# TRAIN

In [ ]:
# from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib

# ==============================
# 1. Load dataset từ CSV
# ==============================
df_train = pd.read_csv("dataset_feedback_students/csv/train.csv")
df_validation = pd.read_csv("dataset_feedback_students/csv/validation.csv")
df_test = pd.read_csv("dataset_feedback_students/csv/test.csv")

print("Kích thước tập train:", df_train.shape)
print("Kích thước tập validation:", df_validation.shape)
print("Kích thước tập test:", df_test.shape)

# ==============================
# 2. Chuẩn hóa label
# ==============================
# Encode sentiment -> số
sentiment_encoder = LabelEncoder()
df_train['sentiment'] = sentiment_encoder.fit_transform(df_train['sentiment'])
df_validation['sentiment'] = sentiment_encoder.transform(df_validation['sentiment'])
df_test['sentiment'] = sentiment_encoder.transform(df_test['sentiment'])

print("Các nhãn sentiment:", list(sentiment_encoder.classes_))

# ==============================
# 3. Chuẩn bị dữ liệu
# ==============================
X_train_text = df_train['sentence'].tolist()
y_train = df_train[['sentiment', 'topic']].values

X_validation_text = df_validation['sentence'].tolist()
y_validation = df_validation[['sentiment', 'topic']].values

X_test_text = df_test['sentence'].tolist()
y_test = df_test[['sentiment', 'topic']].values

# ==============================
# 4. SBERT embedding
# ==============================
print("Đang tải model SBERT...")
sbert_model = SentenceTransformer("sentence-transformers/LaBSE")

print("Đang tạo embedding cho dữ liệu train...")
X_train_embeddings = sbert_model.encode(X_train_text, convert_to_numpy=True, show_progress_bar=True)

print("Đang tạo embedding cho dữ liệu validation...")
X_validation_embeddings = sbert_model.encode(X_validation_text, convert_to_numpy=True, show_progress_bar=True)

print("Đang tạo embedding cho dữ liệu test...")
X_test_embeddings = sbert_model.encode(X_test_text, convert_to_numpy=True, show_progress_bar=True)

print("Kích thước embedding train:", X_train_embeddings.shape)
print("Kích thước embedding validation:", X_validation_embeddings.shape)
print("Kích thước embedding test:", X_test_embeddings.shape)

# ==============================
# 5. Huấn luyện mô hình BalancedRandomForest
# ==============================
brf = BalancedRandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1,
    sampling_strategy="auto",
    replacement=True
)


model = MultiOutputClassifier(brf)
model.fit(X_train_embeddings, y_train)

# ==============================
# 6. Đánh giá mô hình
# ==============================
print("Đánh giá trên tập validation...")
y_validation_pred = model.predict(X_validation_embeddings)

print("\n📌 Kết quả trên tập validation cho nhãn 'sentiment':")
print(classification_report(y_validation[:, 0], y_validation_pred[:, 0], target_names=sentiment_encoder.classes_))

print("\n📌 Kết quả trên tập validation cho nhãn 'topic':")
print(classification_report(y_validation[:, 1], y_validation_pred[:, 1]))

validation_accuracy = (y_validation_pred == y_validation).all(axis=1).mean()
print(f"\n🎯 Độ chính xác tổng thể trên validation: {validation_accuracy:.4f}")

# Đánh giá trên tập test
print("\nĐánh giá trên tập test...")
y_test_pred = model.predict(X_test_embeddings)

print("\n📌 Kết quả trên tập test cho nhãn 'sentiment':")
print(classification_report(y_test[:, 0], y_test_pred[:, 0], target_names=sentiment_encoder.classes_))

print("\n📌 Kết quả trên tập test cho nhãn 'topic':")
print(classification_report(y_test[:, 1], y_test_pred[:, 1]))

test_accuracy = (y_test_pred == y_test).all(axis=1).mean()
print(f"\n🎯 Độ chính xác tổng thể trên test: {test_accuracy:.4f}")

# ==============================
# 7. Vẽ ma trận nhầm lẫn
# ==============================
plt.figure(figsize=(15, 6))

# Sentiment
plt.subplot(1, 2, 1)
cm_sentiment = confusion_matrix(y_test[:, 0], y_test_pred[:, 0])
sns.heatmap(cm_sentiment, annot=True, fmt='d', cmap='Blues',
            xticklabels=sentiment_encoder.classes_,
            yticklabels=sentiment_encoder.classes_)
plt.title('Ma trận nhầm lẫn - Sentiment')
plt.ylabel('Thực tế')
plt.xlabel('Dự đoán')

# Topic
plt.subplot(1, 2, 2)
cm_topic = confusion_matrix(y_test[:, 1], y_test_pred[:, 1])
sns.heatmap(cm_topic, annot=True, fmt='d', cmap='Blues')
plt.title('Ma trận nhầm lẫn - Topic')
plt.ylabel('Thực tế')
plt.xlabel('Dự đoán')

plt.tight_layout()
plt.show()

# # ==============================
# # 8. Lưu mô hình + encoder
# # ==============================
# joblib.dump({"model": model, "sentiment_encoder": sentiment_encoder}, "multioutput_brf_model.pkl")
# print("✅ Đã lưu mô hình vào multioutput_brf_model.pkl")


# ==============================
# 8. Lưu mô hình + encoder + tên SBERT
# ==============================
save_obj = {
    "model": model,                         # mô hình đã huấn luyện
    "sentiment_encoder": sentiment_encoder, # encoder cho sentiment
    "sbert_model_name": "sentence-transformers/LaBSE"  # chỉ cần lưu tên model
}

joblib.dump(save_obj, "multioutput_brf_V2_model.pkl")
print("✅ Đã lưu mô hình vào multioutput_brf_V2_model.pkl")


Kích thước tập train: (11426, 3)
Kích thước tập validation: (1583, 3)
Kích thước tập test: (3166, 3)
Các nhãn sentiment: ['negative', 'neutral', 'positive']
Đang tải model SBERT...
Đang tạo embedding cho dữ liệu train...


Batches:   3%|▎         | 12/358 [00:33<07:09,  1.24s/it] 